# <center>Implementing LDA in Python</center>

## Key Concepts in this Notebook

## Introduction

## Importing the Required Libraries

In [2]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import pandas as pd
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from spacy.lang.nl import Dutch
from spacy.lang.nl.stop_words import STOP_WORDS
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

#vis
import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import string
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag

## Preparing the Data

In [3]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [4]:
stopwords = stopwords.words("dutch")

In [5]:
print(stopwords)

['de', 'en', 'van', 'ik', 'te', 'dat', 'die', 'in', 'een', 'hij', 'het', 'niet', 'zijn', 'is', 'was', 'op', 'aan', 'met', 'als', 'voor', 'had', 'er', 'maar', 'om', 'hem', 'dan', 'zou', 'of', 'wat', 'mijn', 'men', 'dit', 'zo', 'door', 'over', 'ze', 'zich', 'bij', 'ook', 'tot', 'je', 'mij', 'uit', 'der', 'daar', 'haar', 'naar', 'heb', 'hoe', 'heeft', 'hebben', 'deze', 'u', 'want', 'nog', 'zal', 'me', 'zij', 'nu', 'ge', 'geen', 'omdat', 'iets', 'worden', 'toch', 'al', 'waren', 'veel', 'meer', 'doen', 'toen', 'moet', 'ben', 'zonder', 'kan', 'hun', 'dus', 'alles', 'onder', 'ja', 'eens', 'hier', 'wie', 'werd', 'altijd', 'doch', 'wordt', 'wezen', 'kunnen', 'ons', 'zelf', 'tegen', 'na', 'reeds', 'wil', 'kon', 'niets', 'uw', 'iemand', 'geweest', 'andere']


# Getting Data

In [6]:

url = '/Users/dj/Python - UvA/DSP/data/' # Insert your path 
numbers = ["birds", 'elephant', 'fish']
ending = '.txt'

texts = []
for i in numbers:
    p = url + i + ending
    f = open(p,'r')
    text = f.read()
    texts.append(text)
    

data = texts

print (data[2][0:90])

﻿Vissen zijn aquatische, craniate, kieuwdragende dieren die geen ledematen met vingers heb


In [11]:
def preprocess(filename):
    text = filename
    text = text.lower()

    text_p = "".join([char for char in text if char not in string.punctuation])
    
    no_numb = ''.join(c for c in text_p if not c.isdigit())
    
    words = word_tokenize(no_numb)
    
    #stop_words = stopwords.words('dutch')
    filtered_words = [word for word in words if word not in stopwords and len(word) > 2]
    
    lemmatizer = WordNetLemmatizer()
    
    lemmatized = [lemmatizer.lemmatize(word) for word in filtered_words]
    
    return lemmatized

In [12]:
data_words = []
for i in data:
    a = preprocess(i)
    data_words.append(a)

In [13]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100) # min_counts = Minimum counts that it is being taken into consideration
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)


In [14]:
print (data_bigrams_trigrams[2])

['\ufeffvissen', 'aquatische', 'craniate', 'kieuwdragende', 'dieren', 'ledematen', 'vingers', 'definitie', 'vallen', 'levende', 'slijmprikken', 'lampreien', 'kraakbeenvissen', 'beenvissen', 'alsmede', 'diverse', 'uitgestorven', 'verwante', 'groepen', 'ongeveer', 'levende', 'vissoorten', 'straalvinnige', 'vissen', 'behorend', 'klasse', 'actinopterygii', 'waarvan', 'subgroep', 'teleosten', 'behoort', 'vroegste', 'organismen', 'vissen', 'ingedeeld', 'chordaten', 'zachte', 'ledematen', 'eerst', 'verschenen', 'cambrium', 'hoewel', 'echte', 'ruggengraat', 'hadden', 'beschikten', 'notochord', 'waardoor', 'beweeglijker', 'ongewervelde', 'tegenhangers', 'vissen', 'zouden', 'paleozoïcum', 'blijven', 'ontwikkelen', 'grote', 'verscheidenheid', 'vormen', 'aannemen', 'vissen', 'paleozoïcum', 'ontwikkelden', 'uitwendig', 'pantser', 'hen', 'roofdieren', 'beschermde', 'eerste', 'vissen', 'kaken', 'verschenen', 'siluur', 'waarna', 'vissen', 'zoals', 'haaien', 'geduchte', 'zeepredatoren', 'werden', 'plaa

In [15]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow


In [18]:
# id2word = corpora.Dictionary(all_texts)

# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text)
#     corpus.append(new)

# print (corpus[0][0:20])

# word = id2word[[0][:1][0]]
# print (word)

In [16]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=3,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


In [17]:
test_doc = corpus[-1]

vector = lda_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(2, 0.98955035)]
[(2, 0.98955035)]


In [18]:
# lda_model.save("/Users/dj/Documents/GitHub/DSP/topic_modeling_textbook-main/models/test_model.model")

In [19]:
# new_model = gensim.models.ldamodel.LdaModel.load("models/test_model.model")

In [20]:
test_doc = corpus[-1]

vector = lda_model[test_doc] # lda_model can be replaced by new_model
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(2, 0.98955005)]
[(2, 0.98955005)]


## Vizualizing the Data

In [21]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.069196  0.067479       1        1  37.112199
1      0.069925 -0.065623       2        1  34.368848
2     -0.000729 -0.001855       3        1  28.518953, topic_info=               Term      Freq     Total Category  logprob  loglift
200          vogels  8.000000  8.000000  Default  10.0000  10.0000
342       olifanten  8.000000  8.000000  Default   9.0000   9.0000
39           eieren  3.000000  3.000000  Default   8.0000   8.0000
235      afrikaanse  3.000000  3.000000  Default   7.0000   7.0000
201    vogelsoorten  2.000000  2.000000  Default   6.0000   6.0000
104      menselijke  2.000000  2.000000  Default   5.0000   5.0000
125        ongeveer  2.000000  2.000000  Default   4.0000   4.0000
368      slagtanden  2.000000  2.000000  Default   3.0000   3.0000
194        vleugels  2.000000  2.000000  Default   2.0000   2.0000
157         sommige  2.000000  2.000000  Default   1.0000   1.0000
342       olifanten  8.104221  8.675585   Topic1  -3.4545   0.9231
235      afrikaanse  3.013220  3.553468   Topic1  -4.4439   0.8263
368      slagtanden  2.449480  2.983151   Topic1  -4.6510   0.7941
99        mannetjes  1.885482  2.412725   Topic1  -4.9127   0.7447
318           komen  1.885383  2.413631   Topic1  -4.9128   0.7442
240      aziatische  1.884875  2.413765   Topic1  -4.9131   0.7439
279  familiegroepen  1.884256  2.413942   Topic1  -4.9134   0.7435
272    elephantidae  1.884324  2.414402   Topic1  -4.9133   0.7433
376         stieren  1.883643  2.413648   Topic1  -4.9137   0.7433
294           grote  1.883300  2.413679   Topic1  -4.9139   0.7431
200          vogels  8.381395  8.950190   Topic2  -3.3441   1.0024
39           eieren  2.914919  3.453994   Topic2  -4.4003   0.8983
201    vogelsoorten  2.370046  2.903793   Topic2  -4.6072   0.8649
104      menselijke  2.368695  2.903711   Topic2  -4.6078   0.8644
125        ongeveer  2.366220  2.904134   Topic2  -4.6088   0.8632
194        vleugels  1.823670  2.354694   Topic2  -4.8693   0.8125
157         sommige  1.822860  2.354575   Topic2  -4.8697   0.8121
32     dinosauriërs  1.822651  2.354713   Topic2  -4.8698   0.8119
227             één  1.821824  2.355098   Topic2  -4.8703   0.8113
76           hoewel  1.276733  1.805035   Topic2  -5.2258   0.7217
4              alle  0.333344  0.705743   Topic3  -6.3821   0.5045
5        aquatische  0.333344  0.705743   Topic3  -6.3821   0.5045
10          behoort  0.333344  0.705743   Topic3  -6.3821   0.5045
14      belangrijke  0.333344  0.705743   Topic3  -6.3821   0.5045
16        beschouwd  0.333344  0.705743   Topic3  -6.3821   0.5045
19          brengen  0.333344  0.705743   Topic3  -6.3821   0.5045
25     cladistische  0.333344  0.705743   Topic3  -6.3821   0.5045
26     communiceren  0.333344  0.705743   Topic3  -6.3821   0.5045
29          cultuur  0.333344  0.705743   Topic3  -6.3821   0.5045
31           dieren  0.333344  0.705743   Topic3  -6.3821   0.5045
342       olifanten  0.371382  8.675585   Topic3  -6.2741  -1.8964
235      afrikaanse  0.350190  3.553468   Topic3  -6.3328  -1.0626
368      slagtanden  0.345047  2.983151   Topic3  -6.3476  -0.9025
405       vrouwtjes  0.341651  1.845490   Topic3  -6.3575  -0.4321
272    elephantidae  0.343487  2.414402   Topic3  -6.3521  -0.6955
93            leven  0.340909  1.845151   Topic3  -6.3597  -0.4341
187   verschillende  0.340872  1.845143   Topic3  -6.3598  -0.4342
256    bosolifanten  0.340769  1.845114   Topic3  -6.3601  -0.4345
279  familiegroepen  0.342472  2.413942   Topic3  -6.3551  -0.6982
276      essentiële  0.337811  1.275722   Topic3  -6.3688  -0.0742
200          vogels  0.350563  8.950190   Topic3  -6.3318  -1.9853
240      aziatische  0.342032  2.413765   Topic3  -6.3564  -0.6994
294           grote  0.341961  2.413679   Topic3  -6.3566  -0.6996
376         stieren  0.341865  2.413648   Topic3  -6.3569  -0.6999
31

In [ ]:
# Accuracy metric
# Grid search 